Stript to evaluate the predicted labels by comparing corresponding VMAF score of predicted bitrate encoding with VMAF of actual label.

1. import labels

2. import predictions

3. make sure qp0 encodes are available

4. encode qp0 with label and predicted

5. calc vamf of both and compare -> store somehow

In [41]:
import os
import subprocess
import pandas as pd
import numpy as np
import re
import math

In [42]:
### PATHS ###

abs_path='/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/final_data_set'

#location of labels
label_loc=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/aggregated_labels_vmaf92_candidates1-64_v2.csv'

#location of predictions
predictions_loc=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/predictions.csv'



#location of evaluation results from both label and prediction
eval_result_loc_full=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/eval_full.csv'

#location of evaluation results from labels only
eval_result_loc_label=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/eval_label.csv'

#location of evaluation results from prediction only
eval_result_loc_pred=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/eval_pred.csv'

#location of evaluation results given by difference between label and prediction
eval_result_loc_diff=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/eval_diff.csv'

#location of evaluation results current full
eval_result_loc_current=f'/Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/_csv/eval_current_full'

In [43]:
# HELPER METHODS

def extract_vmaf(result_string):
    # Extract the number using regex
    match = re.search(r"VMAF score: (\d+\.\d+)", result_string)

    # Check if a match is found and get the number
    if match:
        vmaf_score = float(match.group(1))
        return vmaf_score 
    else:
        print("VMAF score not found in the string")
        return -101.0

In [44]:
labels=pd.read_csv(label_loc)
predictions=pd.read_csv(predictions_loc)

In [45]:
def eval(directory):

    #init output
    eval_full = pd.DataFrame()
    eval_label = pd.DataFrame()
    eval_pred = pd.DataFrame()
    eval_diff = pd.DataFrame()
    vmaf_label = []
    vmaf_pred = []
    vmaf_diff = [] #pred - label
    scenes = []
    
    for input_vid_name in os.listdir(directory):

        # path to qp0 scenes
        qp0_encodes_loc=f'{abs_path}/{input_vid_name}/qp0_encodes'

        # path to eval encodes
        eval_encodes_loc=f'{abs_path}/{input_vid_name}/eval_encodes'

        #create dir for encodes
        if not os.path.isdir(eval_encodes_loc):
            os.makedirs(eval_encodes_loc)


        #iterate over scenes
        for filename in os.listdir(qp0_encodes_loc):
            print(filename)

            #new dir per scene
            scene_encode_loc = os.path.join(eval_encodes_loc, filename)
            if not os.path.isdir(scene_encode_loc):
                os.makedirs(scene_encode_loc)

            #input vid path
            video_name=f'qp0_{input_vid_name}.mp4'
            qp0_vid_path=f'{qp0_encodes_loc}/{filename}/{video_name}'

            #get label from dataframe
            label_bitrate=labels.loc[labels.Name == f'{filename}','Bitrate'].item()

            if label_bitrate > 0:
            
                #encode with label bitrate
                label_encoded_filename = f'{scene_encode_loc}/label_{label_bitrate}M_{input_vid_name}.mp4'
                encode_command = f'ffmpeg  -v error -i {qp0_vid_path} -c:v libx264 -b:v {label_bitrate}M -preset ultrafast -pass 1 -f null /dev/null &&    \
                                    ffmpeg -v error -i {qp0_vid_path} -c:v libx264 -b:v {label_bitrate}M -preset ultrafast -pass 2 {label_encoded_filename}'
                encode_result = subprocess.run(encode_command, capture_output=True, text=True, shell=True)
                print(encode_result)


                #get prediction from dataframe
                predicted_bitrate=predictions.loc[labels.Name == f'{filename}','Prediction'].item()

                #encode with predicted bitrate
                prediction_encoded_filename = f'{scene_encode_loc}/predicted_{label_bitrate}M_{input_vid_name}.mp4'
                encode_command = f'ffmpeg  -v error -i {qp0_vid_path} -c:v libx264 -b:v {predicted_bitrate}M -preset ultrafast -pass 1 -f null /dev/null &&    \
                                    ffmpeg -v error -i {qp0_vid_path} -c:v libx264 -b:v {predicted_bitrate}M -preset ultrafast -pass 2 {prediction_encoded_filename}'
                encode_result = subprocess.run(encode_command, capture_output=True, text=True, shell=True)
                print(encode_result)

                # calc vmaf of label
                vmaf_command = f'ffmpeg -i {label_encoded_filename} -i {qp0_vid_path} -filter_complex libvmaf -f null -'
                vmaf_result = subprocess.run(vmaf_command, capture_output=True, text=True, shell=True)
                vmaf_score_label=extract_vmaf(str(vmaf_result))
                print('LABEL VMAF SCORE: ', vmaf_score_label)

                # calc vmaf of prediction
                vmaf_command = f'ffmpeg -i {prediction_encoded_filename} -i {qp0_vid_path} -filter_complex libvmaf -f null -'
                vmaf_result = subprocess.run(vmaf_command, capture_output=True, text=True, shell=True)
                vmaf_score_pred=extract_vmaf(str(vmaf_result))
                print('PREDICTION VMAF SCORE: ', vmaf_score_pred)

            else:
                vmaf_score_label = -101.0
                vmaf_score_pred = -101.0

            scenes.append(filename)
            vmaf_label.append(vmaf_score_label)
            vmaf_pred.append(vmaf_score_pred)

            vmaf_score_diff = 0
            if vmaf_score_pred == -101.0 or vmaf_score_label == -101.0:
                vmaf_score_diff = -101.0
            elif vmaf_score_pred != vmaf_score_label:
                vmaf_score_diff = vmaf_score_pred - vmaf_score_label
            vmaf_diff.append(vmaf_score_diff)

        current_full = pd.DataFrame()
        current_full.assign(Name=scenes, VMAF_LABEL=vmaf_label, VMAF_PRED=vmaf_pred)
        current_full.to_csv(f'{eval_result_loc_current}_{input_vid_name}.csv')
    
    eval_full.assign(Name=scenes, VMAF_LABEL=vmaf_label, VMAF_PRED=vmaf_pred)
    eval_full.to_csv(eval_result_loc_full)

    eval_label.assign(Name=scenes, VMAF_LABEL=vmaf_label)
    eval_label.to_csv(eval_result_loc_label)

    eval_pred.assign(Name=scenes, VMAF_PRED=vmaf_pred)
    eval_pred.to_csv(eval_result_loc_pred)

    eval_diff.assign(Name=scenes, VMAF_DIFF=vmaf_diff)
    eval_diff.to_csv(eval_result_loc_diff)



eval(abs_path)
        

FourPeople_1280x720_60-Scene-001
CompletedProcess(args='ffmpeg  -v error -i /Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/final_data_set/FourPeople_1280x720_60/qp0_encodes/FourPeople_1280x720_60-Scene-001/qp0_FourPeople_1280x720_60.mp4 -c:v libx264 -b:v 4.0M -preset ultrafast -pass 1 -f null /dev/null &&                                        ffmpeg -v error -i /Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/final_data_set/FourPeople_1280x720_60/qp0_encodes/FourPeople_1280x720_60-Scene-001/qp0_FourPeople_1280x720_60.mp4 -c:v libx264 -b:v 4.0M -preset ultrafast -pass 2 /Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/final_data_set/FourPeople_1280x720_60/eval_encodes/FourPeople_1280x720_60-Scene-001/label_4.0M_FourPeople_1280x720_60.mp4', returncode=0, stdout='', stderr='')
CompletedProcess(args='ffmpeg  -v error -i /Volumes/T7/deep_encode_dataset/DATASET_DEEP_ENCODE_2/final_data_set/FourPeople_1280x720_60/qp0_encodes/FourPeople_1280x720_60-Scene-001/qp0_FourPeople_12

ValueError: can only convert an array of size 1 to a Python scalar